In [1]:
import nltk
import numpy as py
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

import re
import numpy as np
import pandas as pd
from pprint import pprint

# spacy for lemmatization
import spacy
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [13]:
df = pd.read_csv("../../Youth_Recidivism_Project/Dataset/DYS_Casenotes_16Oct20_merged_final_bydischarge.csv")
#df['WHO_WAS_CONTACTED'][0], df['COMMENTS_CLOB']
df['STAFF_NOTES'][df.STAFF_NOTES != df.STAFF_NOTES] = ""
df['COMMENTS_CLOB'][df.COMMENTS_CLOB != df.COMMENTS_CLOB] = ""
df['STAFF_REC'][df.STAFF_REC != df.STAFF_REC] = ""
print(df['STAFF_NOTES'][df.STAFF_NOTES != df.STAFF_NOTES])

Series([], Name: STAFF_NOTES, dtype: object)


/Users/lw/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1,44,46) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/lw/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/lw/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/lw/ana

In [14]:
df['total_notes'] = df['COMMENTS_CLOB'] + df['STAFF_NOTES'] + df['STAFF_REC']
na_values = df[df.total_notes == ""].index.tolist()
print(df.shape)
df = df.drop(na_values, axis = 0)

(146353, 51)


In [15]:
df['WHO_WAS_CONTACTED']
df.iloc[115,:]
arr = []
df.iloc[115,16]
for i in range(df.shape[0]):
    if df.iloc[i,16] == 'Staffing (CW use only)':
        arr.append(0)
    elif type(df.iloc[i,19]) == float:
        arr.append(3)
    elif 'Client' in df.iloc[i,19]:
        arr.append(1)
    elif 'Client' not in df.iloc[i,19] and 'Collateral' in df.iloc[i,19]:
        arr.append(2)
    else:
        arr.append(3)

In [16]:
df['label'] = np.array(arr)
print(df)

        Unnamed: 0     MID FirstCommit_2 DischargeDate_2  CommitDays  \
0              0.0  346868     2009/12/1        2016/7/5      2408.0   
1              1.0  346868     2009/12/1        2016/7/5      2408.0   
2              2.0  346868     2009/12/1        2016/7/5      2408.0   
3              3.0  346868     2009/12/1        2016/7/5      2408.0   
4              4.0  346868     2009/12/1        2016/7/5      2408.0   
...            ...     ...           ...             ...         ...   
146348    146345.0  509896     2019/10/4       2019/12/6        63.0   
146349    146346.0  509896     2019/10/4       2019/12/6        63.0   
146350    146347.0  509896     2019/10/4       2019/12/6        63.0   
146351    146348.0  509896     2019/10/4       2019/12/6        63.0   
146352    146349.0  509896     2019/10/4       2019/12/6        63.0   

        monthsofstay  cnpermnth  numcasenote  numcaseworkers  numcasenoteCW  \
0          79.210526   0.164120         13.0            

In [17]:
df.to_pickle("../../Youth_Recidivism_Project/Dataset/train.pickle")